In [23]:
!pip install transformers datasets torch accelerate


In [24]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset


In [25]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [26]:
tokenizer.pad_token = tokenizer.eos_token


In [27]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:10%]")


In [28]:
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


In [29]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [30]:

dataset = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]



In [31]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-Rak",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=8,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.719104
2,1.807600,1.671347


TrainOutput(global_step=618, training_loss=1.7574470297804157, metrics={'train_runtime': 209.1849, 'train_samples_per_second': 47.384, 'train_steps_per_second': 2.954, 'total_flos': 161220095115264.0, 'train_loss': 1.7574470297804157, 'epoch': 2.9878934624697338})

In [33]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 1.6713467836380005, 'eval_runtime': 1.8311, 'eval_samples_per_second': 200.978, 'eval_steps_per_second': 100.489, 'epoch': 2.9878934624697338}


In [34]:
def generate_text(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids=input_ids,
        num_beams=5,
        max_length=200,
        no_repeat_ngram_size=2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [35]:
prompt = "Once upon a time in a distant kingdom,"
generated_text = generate_text(prompt)
print(f"Generated Text: {generated_text}")



Generated Text: Once upon a time in a distant kingdom, the king of the Kingdom of Cagliari , the son of his father , died of cancer . He was buried in the tomb of a king , and his tomb was dedicated to his son , who died on the way home from the funeral . The tomb itself was named after him by his mother , whom he had known since he was a child . His father was also the patron saint of many other deities , such as the god Shiva , which was worshipped by the gods Shiva and the goddess Shiva . Shiva was the first god to be worshiped by a god , but was not the only one to worship it . In his lifetime , Shiva had been venerated by many gods , including the sun , moon , sun and earth , as well as by gods of all ages . As a result of this veneration , he became a symbol of power and power in all aspects of life . 



In [36]:
model.save_pretrained("./gpt2-finetuned-Rakesh")
tokenizer.save_pretrained("./gpt2-finetuned-Rakesh")

('./gpt2-finetuned-Rakesh/tokenizer_config.json',
 './gpt2-finetuned-Rakesh/special_tokens_map.json',
 './gpt2-finetuned-Rakesh/vocab.json',
 './gpt2-finetuned-Rakesh/merges.txt',
 './gpt2-finetuned-Rakesh/added_tokens.json',
 './gpt2-finetuned-Rakesh/tokenizer.json')

In [37]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 1.6713467836380005, 'eval_runtime': 2.5847, 'eval_samples_per_second': 142.379, 'eval_steps_per_second': 71.189, 'epoch': 2.9878934624697338}


In [38]:
prompt = "Virat Kohli is a Cricketer,"
generated_text = generate_text(prompt)
print(f"Generated Text: {generated_text}")


Generated Text: Virat Kohli is a Cricketer, who has won four ODIs in his career . He was named to the All India team for the second time in a row by the World Twenty20 team . In his first match of the season against Sri Lanka , he scored a century @-@ century wicket in the first over of India 's first Test match against South Africa . 



# New Section